In [1]:
import pandas as pd
import json
import requests
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

## Ekstrak Data

In [2]:
# Mengambil data dari URL
response = requests.get("https://gist.githubusercontent.com/nadirbslmh/93b62fdcfa694d4ec07bed9b3c94e401/raw/c07971341361e23fd4f3a880437c4d8e4ebcfafc/stock_trades.json")
data = response.json()

# Membuat DataFrame dari data JSON
df = pd.DataFrame(data)

# Menampilkan lima baris pertama DataFrame
print(df.head())


   trade_id stock_symbol trade_date  trade_time  trade_price  trade_volume  \
0         1        GOOGL   1/5/2017           3       675.51        501030   
1         2         NVDA  3/27/2017          13       559.39         51823   
2         3         NFLX  11/7/2015           0       811.33        684430   
3         4         MSFT  5/29/2017          22       231.21        237465   
4         5          JPM  11/8/2012          16       699.15        800344   

   buyer_id  seller_id trade_type currency  
0         1          1       sell      EUR  
1         2          2        buy      ZAR  
2         3          3       sell      ZWL  
3         4          4        buy      IDR  
4         5          5        buy      HRK  


## Transformasi Data

In [6]:
transaksi_saham = df[(df['stock_symbol'].isin(['GOOGL', 'AMZN', 'MSFT', 'AAPL'])) & (df['trade_price'] > 500)]

# Menampilkan lima baris pertama DataFrame hasil transformasi
print(transaksi_saham.head())

filename = 'transaksi_saham.csv'
transaksi_saham.to_csv(filename, index=False)

    trade_id stock_symbol  trade_date  trade_time  trade_price  trade_volume  \
0          1        GOOGL    1/5/2017           3       675.51        501030   
7          8         AMZN  11/27/2017          16       827.92        384783   
15        16         MSFT    1/1/2020           5       829.56        131961   
18        19         MSFT   11/9/2015          21       753.17        994135   
34        35         MSFT   7/12/2020           1       617.64        735687   

    buyer_id  seller_id trade_type currency  
0          1          1       sell      EUR  
7          8          8        buy      CNY  
15        16         16       sell      CUP  
18        19         19       sell      CNY  
34        35         35        buy      RUB  


## LOAD DATA

In [7]:
cred = credentials.Certificate(r"C:\codes\de_muhammad-dzikri-rizaldi\19_Data_Engineering_in_The_Cloud\praktikum\serviceAccountKey.json")
firebase_admin.initialize_app(cred, {"storageBucket": "data-engineering-in-the-cloud.appspot.com"})
bucket = storage.bucket()

df = pd.read_csv("transaksi_saham.csv")
df.to_parquet("transaksi_saham.parquet")

blob = bucket.blob("transaksi_saham.parquet")
blob.upload_from_filename("transaksi_saham.parquet")